<a href="https://colab.research.google.com/github/201524495/2021_FSRnT/blob/main/kerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# 모델 정의
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])
  
  return model

In [ ]:
# 튜너 인스턴스화 & 하이퍼튜닝 수행
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

INFO:tensorflow:Reloading Oracle from existing project my_dir/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from my_dir/intro_to_kt/tuner0.json


In [ ]:
# 훈련 단계가 끝나면 훈련 결과를 지우도록 callback
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
# 하이퍼파라미터 검색
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 512 and the optimal learning rate for the optimizer
is 0.0001.



In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 15.4194 - accuracy: 0.7077 - val_loss: 5.4741 - val_accuracy: 0.7945
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 4.3706 - accuracy: 0.8164 - val_loss: 3.5886 - val_accuracy: 0.8175
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 3.1305 - accuracy: 0.8311 - val_loss: 4.1739 - val_accuracy: 0.8142
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 2.4682 - accuracy: 0.8464 - val_loss: 2.8966 - val_accuracy: 0.8119
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 1.9821 - accuracy: 0.8498 - val_loss: 3.0327 - val_accuracy: 0.8060
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 1.6549 - accuracy: 0.8623 - val_loss: 2.1915 - val_accuracy: 0.8340
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 1.3508 - accuracy: 0.8690 - val_loss: 1.7884 - val_accuracy